In [1]:
import sys # aby importy z dołu działały
sys.path.append(r"C:\Users\piotr\OneDrive\____Moje-MOJE\MyProjects_4Fun\projects\World of Warcraft\python-etl")

In [2]:
from sqlalchemy import text, bindparam
from moduly.db_core import utworz_engine_do_db, _czy_duplikat
from moduly.services_persist_wynik import przefiltruj_dane_misji, zapisz_misje_dialogi_ai_do_db
from moduly.ai_gemini import zaladuj_api_i_klienta, instrukcja_tlumacz, instrukcja_redaktor
from moduly.utils import sklej_warunki_w_WHERE
from sqlalchemy.exc import IntegrityError
from scraper_wiki_main import parsuj_misje_z_url
from google import genai
import json
import zlib
import base64
from bs4 import BeautifulSoup
import pandas as pd
from pprint import pprint

In [3]:
# chce stworzyc proces, w ktorym bede mogl zatwierdzac tlumaczenia
# potrzebuje do tego misje w podziale na grupy:
#    a. oryinal,
#    b. przetlumaczone,
#    c. zredagowane

# oraz potrzebuje do tego wypowiedzi w podziale na grupy:
#    a. oryinal,
#    b. przetlumaczone,
#    c. zredagowane

# oraz tlumaczenia NPCow, Slow kluczowych
# oraz sprawdzic kazde slowo w misji/wypowiedzi czy wystepuje w Slowach kluczowych (cross check)

# calosc potrzebuje wyrzucic do excela w takim ukladzie:
#    a. 

In [4]:
silnik = utworz_engine_do_db()

In [ ]:
def stworz_excele_do_zatwierdzenia_tlumaczen(silnik, kraina = None, fabula = None, dodatek = None, sciezka = None):

    warunki_sql = sklej_warunki_w_WHERE(kraina, fabula, dodatek)

    q_select_misje_id = text(f"""
        SELECT m.MISJA_ID_MOJE_PK
        FROM dbo.MISJE AS m
        WHERE 1=1
        AND m.MISJA_ID_MOJE_PK <> 123456789
                        
        {warunki_sql}

        ORDER BY m.MISJA_ID_MOJE_PK ASC
        ;
    """)

    q_select_tytul = text(f"""
        SELECT MISJA_ID_MOJE_PK, MISJA_TYTUL_EN, MISJA_TYTUL_PL, NAZWA_LINII_FABULARNEJ_EN, NAZWA_LINII_FABULARNEJ_PL
        FROM dbo.MISJE
        WHERE MISJA_ID_MOJE_PK IN :misje_id
    ;
    """).bindparams(bindparam("misje_id", expanding=True))

    q_select_misje_dialogi = text("""
        SELECT MISJA_ID_MOJE_FK, SEGMENT, STATUS, NR_BLOKU_DIALOGU, NR_WYPOWIEDZI, TRESC
        FROM [dbo].[DIALOGI_STATUSY]
        WHERE MISJA_ID_MOJE_FK IN :misje_id
    """).bindparams(bindparam("misje_id", expanding=True))

    q_select_misje_tresci = text("""
        SELECT MISJA_ID_MOJE_FK AS MISJA_ID, SEGMENT, PODSEGMENT, STATUS, NR AS NR_BLOKU, TRESC
        FROM dbo.MISJE_STATUSY
        WHERE MISJA_ID_MOJE_FK IN :misje_id
    """).bindparams(bindparam("misje_id", expanding=True))
    
    parametry = {
        "kraina_en": kraina, 
        "fabula_en": fabula, 
        "dodatek_en": dodatek
    }

    with silnik.connect() as conn:
        misje = conn.execute(q_select_misje_id, parametry).scalars().all()
        tytuly = conn.execute(q_select_tytul, {"misje_id": misje}).mappings().all()
        misje_dialogi = conn.execute(q_select_misje_dialogi, {"misje_id": misje}).mappings().all()
        misje_tresci = conn.execute(q_select_misje_tresci, {"misje_id": misje}).mappings().all()

    df_tytuly = pd.DataFrame(tytuly)
    df_misje_dialogi = pd.DataFrame(misje_dialogi)
    df_misje_tresci = pd.DataFrame(misje_tresci)

    nowe_naglowki = {
        "MISJA_ID_MOJE_PK": "MISJA_ID",
        "MISJA_ID_MOJE_FK": "MISJA_ID",
        "value": "TRESC",
        "NR_WYPOWIEDZI": "NR_WYP",
        "NR_BLOKU_DIALOGU": "NR_BLOKU"
    }
    
    nowe_wiersze = {
        "MISJA_TYTUL_EN": "0_ORYGINAŁ",
        "MISJA_TYTUL_PL": "3_ZATWIERDZONO"
    }

    mapping_segmentow = {
        "TYTUL": 1, "CEL": 2, "TREŚĆ": 3, "POSTĘP": 4, "ZAKOŃCZENIE": 5, "NAGRODY": 6, "DYMEK": 7, "GOSSIP": 8
    }
    
    kolejnosc_kolumn_koniec = [
        "MISJA_ID", "STATUS", "SEGMENT", "PODSEGMENT", "ID_SEGMENTU", "NR_BLOKU", "NR_WYP", "TRESC"
    ]

    df_tytuly = (
        df_tytuly.melt(
            ["MISJA_ID_MOJE_PK"], 
            value_vars=["MISJA_TYTUL_EN", "MISJA_TYTUL_PL"], 
            var_name="STATUS")
        .rename(columns=nowe_naglowki)
        .replace(nowe_wiersze)
        .assign(SEGMENT = "TYTUL", PODSEGMENT = "", NR_BLOKU = 1, NR_WYP = 1)
        )
    
    df_misje_dialogi = (
        df_misje_dialogi
        .rename(columns=nowe_naglowki)
        .assign(PODSEGMENT = "")
    )

    df_misje_tresci = (
        df_misje_tresci
        .rename(columns=nowe_naglowki)
        .assign(NR_WYP = 1)
    )

    df_polaczone = (
            pd.concat([df_tytuly, df_misje_dialogi, df_misje_tresci])
            .assign(
                ID_SEGMENTU = lambda x: x["SEGMENT"].map(mapping_segmentow)
            )
            .sort_values(by=["MISJA_ID", "STATUS", "ID_SEGMENTU", "NR_BLOKU", "NR_WYP"])
            .reset_index(drop=True)
            [kolejnosc_kolumn_koniec]
        )
    
    with pd.ExcelWriter(sciezka, engine="xlsxwriter") as zapis: # type: ignore
            df_polaczone.to_excel(zapis, sheet_name="Tlumaczenia", index=False)
            
            arkusz = zapis.sheets["Tlumaczenia"]
            
            format_bazowy = zapis.book.add_format({ # type: ignore
                "bg_color": "black",
                "font_color": "white",
                "align": "center",
                "valign": "vcenter"
            })
            
            format_zawijania = zapis.book.add_format({ # type: ignore
                "bg_color": "black",
                "font_color": "white",
                "align": "center",
                "valign": "vcenter",
                "text_wrap": True
            })
            
            arkusz.set_column("A:XFD", None, format_bazowy)
            
            arkusz.set_column("A:G", 15, format_bazowy)
            arkusz.set_column("H:H", 80, format_zawijania)

    return df_polaczone

In [92]:
stworz_excele_do_zatwierdzenia_tlumaczen(
    silnik, 
    fabula="The Cult Within",
    sciezka=r"C:\Users\piotr\OneDrive\____Moje-MOJE\MyProjects_4Fun\projects\World of Warcraft\excel-zatwierdzenia\The Cult Within.xlsx"
    )

,MISJA_ID,STATUS,SEGMENT,PODSEGMENT,ID_SEGMENTU,NR_BLOKU,NR_WYP,TRESC
0,7377,0_ORYGINAŁ,TYTUL,,1,1,1,Avoiding Blame
1,7377,0_ORYGINAŁ,CEL,PODRZĘDNY_CEL,2,1,1,Suspicious Citizens dispersed (6)
2,7377,0_ORYGINAŁ,CEL,GŁÓWNY_CEL,2,1,1,Disperse 6 Suspicious Citizens in Stormwind.
3,7377,0_ORYGINAŁ,TREŚĆ,NaN,3,1,1,Apocalyptic threats have taken many forms in A...
4,7377,0_ORYGINAŁ,TREŚĆ,NaN,3,2,1,I fear that people are falling into hopelessne...
...,...,...,...,...,...,...,...,...
960,7943,2_ZREDAGOWANO,GOSSIP,,8,6,4,"Po czwarte: pław się w agonii innych, gdy tylk..."
961,7943,2_ZREDAGOWANO,GOSSIP,,8,6,5,"Po piąte: walcz o swoją pozycję, poświęcając t..."
962,7943,2_ZREDAGOWANO,GOSSIP,,8,6,6,Po szóste: strzeż chwały Ostrza i pod żadnym p...
963,7943,2_ZREDAGOWANO,GOSSIP,,8,6,7,<Lista ciągnie się przez wiele kolejnych stron.>


In [ ]:
# wynik = stworz_excele_do_zatwierdzenia_tlumaczen(silnik, fabula="The Cult Within")
# wynik.to_excel(r"C:\Users\piotr\OneDrive\____Moje-MOJE\MyProjects_4Fun\projects\World of Warcraft\aaa.xlsx")